## Resources

* https://github.com/benkay86/nom-tutorial
* https://blog.logrocket.com/parsing-in-rust-with-nom/
* https://iximiuz.com/en/posts/rust-writing-parsers-with-nom/
* https://developerlife.com/2023/02/20/guide-to-nom-parsing/
* Extended error: https://github.com/rust-bakery/nom/blob/main/examples/string.rs#L168

In [14]:
:dep nom
:dep nom-supreme

## Hex color parser

In [71]:
use nom;
use nom_supreme::error::ErrorTree;
use nom_supreme::context;
use nom_supreme::ParserExt;
use nom_supreme::tag::complete::tag;

In [59]:
#[derive(Debug, PartialEq)]
struct Color {
    red: u8,
    green: u8,
    blue: u8,
}

In [60]:
fn parse_hex_seg(input: &str) -> nom::IResult<&str, u8> {
    nom::combinator::map_res(
        nom::bytes::complete::take_while_m_n(2,2, 
            |it: char| it.is_ascii_hexdigit()),
      |it: &str| u8::from_str_radix(it, 16),
    )(input)
}
        

In [61]:
let _ = match parse_hex_seg("Ax") {
    Ok((left, parsed)) => println!("left: {left}, parsed {parsed}"),
    Err(e) => println!("{:?}", e),
};

Error(Error { input: "Ax", code: TakeWhileMN })


In [65]:
fn hex_color_no_alpha(input: &str) -> nom::IResult<&str, Color> {
    let (input, _) = nom::bytes::complete::tag("#")(input)?;
//     nom::multi::count(parse_hex_seg, 3)(input).map(
//        |(i, v)| (i, Color{red: v[0], green: v[1], blue: v[2]}))
     nom::sequence::tuple((parse_hex_seg,
                        parse_hex_seg,
                        parse_hex_seg))(input).map(
        |(i, v)| (i, Color{red: v.0, green: v.1, blue: v.2}))
}

In [63]:
let _ = match hex_color_no_alpha("0A21AA") {
    Ok((_, Color{red: r, green: g, blue: b})) => println!("r: {r}, g: {g}, b: {b}"),
    Err(e) => println!("{:?}", e),
};

Error(Error { input: "0A21AA", code: Tag })


In [101]:
fn parse_hex_seg2(input: &'static str) -> nom::IResult<&str, u8, ErrorTree<&str>> {
    nom::combinator::map_res(
        nom::bytes::complete::take_while_m_n(2,2, 
                    |it: char| it.is_ascii_hexdigit())
                    .context("Wrong digit format").context(&input[..2]),
                    |it: &str| u8::from_str_radix(it, 16),
    )(input)
}
fn hex_color_no_alpha2(input: &'static str) -> nom::IResult<&str, Color, ErrorTree<&str>> {
    let (input, _) = nom_supreme::tag::complete::tag("#")(input)?;
//     nom::multi::count(parse_hex_seg, 3)(input).map(
//        |(i, v)| (i, Color{red: v[0], green: v[1], blue: v[2]}))
    let (input, (red, green, blue)) = 
     nom::sequence::tuple((parse_hex_seg2,
                        parse_hex_seg2,
                        parse_hex_seg2))(input)?;
    Ok((input, Color{red, green, blue}))
}

In [107]:
let _ = match hex_color_no_alpha2("#0a21AA") {
    Ok((_, Color{red: r, green: g, blue: b})) => println!("r: {r}, g: {g}, b: {b}"),
    Err(e) => println!("{:?}", e),
};

r: 10, g: 33, b: 170


In [108]:
dbg!(hex_color_no_alpha2("#0A2$AA"));

[src/lib.rs:52] hex_color_no_alpha2("#0A2$AA") = Err(
    Error(
        Stack {
            base: Base {
                location: "2$AA",
                kind: Kind(
                    TakeWhileMN,
                ),
            },
            contexts: [
                (
                    "2$AA",
                    Context(
                        "Wrong digit format",
                    ),
                ),
                (
                    "2$AA",
                    Context(
                        "2$",
                    ),
                ),
            ],
        },
    ),
)


## Custom error type (pq parser)

https://iximiuz.com/en/posts/rust-writing-parsers-with-nom/

In [109]:
use nom::{
    branch::alt,
    bytes::complete::{tag, tag_no_case},
    character::complete::{alpha1, alphanumeric1},
    combinator::recognize,
    multi::many0,
    sequence::pair,
    IResult
};

In [196]:
use nom::combinator::recognize;
use nom::character::complete::{char, alpha1};
use nom::sequence::separated_pair;
use nom::{Err, error::ErrorKind};
use nom::IResult;
use nom::character::complete::space0;
use nom::sequence::delimited;
use nom::bytes::complete::is_not;
use nom::combinator::value;
use nom::sequence::preceded;
use nom::sequence::tuple;
use nom::multi::separated_list1;

{
let mut parser = recognize(separated_pair(alpha1, char(','), alpha1));

assert_eq!(parser("abcd,efgh"), Ok(("", "abcd,efgh")));
assert_eq!(parser("abcd;"),Err(Err::Error((";", ErrorKind::Char))));
}


()

In [197]:
pub fn identifier(input: &str) -> IResult<&str, &str> {
    // [a-zA-Z_][a-zA-Z0-9_]*
    let (rest, m) = recognize(pair(
        alt((alpha1, tag("_"))),
        many0(alt((alphanumeric1, tag("_")))),
    ))(input)?;
    Ok((rest, m))
}

In [198]:
pub fn string_literal(input: &str) -> IResult<&str, &str> {
    let (rest, m) = recognize(
        delimited(char('"'), many0(is_not("\"")), char('"'))
    )(input)?;
    Ok((rest, &m[1..m.len() - 1]))
}

In [199]:
string_literal("\"ciao\"")

Ok(("", "ciao"))

In [200]:
#[derive(Clone, Copy, Debug, PartialEq)]
pub enum MatchOp {
    Eql,   // ==
    Neq,   // !=
    EqlRe, // =~
    NeqRe, // !~
}

In [201]:
pub fn match_op(input: &str) -> IResult<&str, MatchOp> {
    alt((
        value(MatchOp::Eql, tag("==")),
        value(MatchOp::Neq, tag("!=")),
        value(MatchOp::EqlRe, tag("=~")),
        value(MatchOp::NeqRe, tag("!~")),
    ))(input)
}


In [182]:
#[derive(Clone, Copy, Debug, PartialEq)]
pub struct LabelMatch<'a> {
    pub label: &'a str,
    pub value: &'a str,
    pub op: MatchOp,
}

In [202]:
pub fn label_match(input: &str) -> IResult<&str, LabelMatch> {
    let (rest, (label, op, value)) = 
        tuple((identifier, 
               preceded(space0, match_op), 
               preceded(space0, string_literal)))(input)?;
    Ok((rest, LabelMatch{label, value, op}))
}

In [203]:
#[derive(Clone, Debug, PartialEq)]
pub struct VectorSelector<'a> {
    pub metric: &'a str,
    pub labels: Vec<LabelMatch<'a>>,
}

In [217]:
pub fn vector_selector(input: &str) -> IResult<&str, VectorSelector> {
    //@todo: nom supreme
//     let (rest, metric) = identifier(input)?;
//     let (rest, _) = char('{')(rest)?;
//     let (rest, labels) = separated_list0(char(','), label_match)(rest)?;
//     let (rest, _) = char('}')(rest)?;
//     Ok((rest, VectorSelector { metric, labels }))
    let (rest, (metric, _, labels, _)) = tuple((
        identifier,
        preceded(space0, char('{')),
        preceded(space0, separated_list1(char(','), label_match)),
        preceded(space0, char('}')),
        ))(input)?;
    Ok((rest, VectorSelector{ metric, labels}))
}

In [222]:
println!("{:?}", vector_selector(r#"foo {bar ==  "baz",qux!="123",abc!~"mux"}"#));

Ok(("", VectorSelector { metric: "foo", labels: [LabelMatch { label: "bar", value: "baz", op: Eql }, LabelMatch { label: "qux", value: "123", op: Neq }, LabelMatch { label: "abc", value: "mux", op: NeqRe }] }))


### Better errors

In [223]:
:dep nom_locate

In [225]:
use nom_locate;

In [226]:
use nom_locate::LocatedSpan;

In [232]:
type Span<'a> = LocatedSpan<&'a str>;
type Res<'a, O> = nom::IResult<Span<'a>, O>;